In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
from datetime import datetime
from IPython.display import display
import json
import kaleido  # needed for saving plots
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.model_selection import KFold
from tqdm import tqdm

from regression import *

# Parameters

In [3]:
benchmarks_to_analyze = ['MMLU', 'GPQA', 'GSM1k', 'BBH']
# Llama ablations
exclude_big_llama = False
exclude_all_llamas = False

bench_is_accuracy = {'MMLU': True, 'BBH': True, 'GSM1k': True, 'GPQA': True, 'LMSys Elo': False, 'SEAL Coding': False, 'SEAL Math': False}
benchmark_noise_tolerance = 0.02  # in nats - default of 0.02 roughly means that error rates have a relative diff of <2%
plot_log_error = False  # Whether to plot the negative log of error rate instead of accuracy. This only applies to performance vs. compute plots.
non_suspects_only = True  # Whether to only include not-suspicious benchmark scores in the analysis
trusted_only = False  # Whether to only include actively trusted benchmark scores in the analysis (more strict)
old_models_only = False  # Whether to only include models released before November 2023
new_models_only = False  # Whether to only include models released in or after November 2023
bootstrap_compute_performance_regression = False  # Whether to bootstrap the compute-performance regression
save = True  # Whether to save plots and results to disk

color_map = {'Open': 'blue', 'Closed': 'darkorange'}

In [4]:
results_dir = 'results/benchmark/4Nov/'
os.makedirs(results_dir, exist_ok=True)
os.makedirs(f"{results_dir}/plot_data", exist_ok=True)

In [5]:
rng = np.random.default_rng(seed=42)

In [6]:
def save_plot(fig, folder, filename, extensions=['png', 'svg', 'pdf'], scale=2):
    prefix = ''
    if exclude_all_llamas:
        prefix = 'all_llamas_excluded_'
    elif exclude_big_llama:
        prefix = 'big_llama_excluded_'
        
    for ext in extensions:
        fig.write_image(folder + prefix + filename + '.' + ext, scale=scale)
    fig.write_html(folder + prefix + filename + '.html')

# Prepare data

In [7]:
# data_path = "https://docs.google.com/spreadsheets/d/1etu9rXcME0uUA-S2ANA8bsfQbIZgNu-8NxqFGQdDIzQ/export?format=csv&gid=1305280917#gid=1305280917"
data_path = "data/benchmarks_with_model_accessibility.csv"
df = pd.read_csv(data_path)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df.head(30)

,System,Model size (parameters),Active Parameters,Dataset size,Date,Open/Closed,Training compute (FLOP),Training compute notes,BBH,GPQA,MMLU,HELM MMLU,SEAL Coding,SEAL Instruction Following,SEAL Math,LMSys Elo,LMSys Elo Notes,LMSys Elo 95% CI,BBH Notes,GPQA Notes,MMLU Notes,HELM MMLU Notes,Trust in benchmark results,Trust notes
0,BLOOM-176B,1.760000e+11,"176,000,000,000",3.900000e+11,2022-07-11,Open,4.120000e+23,NaN,0.449100,NaN,0.3913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,BloombergGPT,5.000000e+10,"50,000,000,000",7.080000e+11,2023-03-30,Closed,2.120000e+23,NaN,0.419700,NaN,0.3918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,Camelidae-8x34B,NaN,NaN,NaN,2024-01-05,Open,NaN,NaN,NaN,NaN,0.7560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,ChatGLM-6B,6.000000e+09,"6,000,000,000",NaN,2023-03-01,Open,NaN,NaN,0.187300,NaN,NaN,NaN,NaN,NaN,NaN,880.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,ChatGLM2-12B-base,1.200000e+10,"12,000,000,000",NaN,2023-06-25,Open,NaN,NaN,0.360200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
5,ChatGLM2-6B-base,6.000000e+09,"6,000,000,000",NaN,2023-06-25,Open,NaN,NaN,0.336800,NaN,NaN,NaN,NaN,NaN,NaN,924.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
6,ChatGLM3-6B,6.000000e+09,"6,000,000,000",NaN,2023-10-27,Open,5.040000e+22,NaN,0.661000,NaN,NaN,NaN,NaN,NaN,NaN,955.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
7,Chinchilla 70B,NaN,"70,000,000,000",NaN,2022-03-29,Closed,5.760000e+23,NaN,NaN,NaN,0.6750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
8,Claude 2,NaN,NaN,NaN,2023-07-11,Closed,NaN,NaN,NaN,0.353,0.7850,NaN,NaN,NaN,NaN,1132.0,NaN,NaN,NaN,Epoch evaluation,"Actually CoT, so probably an overestimate. HEL...",NaN,0,NaN
9,Claude 2.1,NaN,NaN,NaN,2023-11-21,Closed,NaN,NaN,NaN,0.361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Epoch evaluation,NaN,NaN,0,Doesn't perform worse on GSM1k relative to GSM8k


In [8]:
# gsm1k_data_path = "https://docs.google.com/spreadsheets/d/1KYp4h3urj-698IE9bR7n1ctuH1iyCAQ5pTZIqQ_qs9g/export?format=csv"
gsm1k_data_path = "data/gsm1k_with_model_accessibility.csv"
gsm1k_df = pd.read_csv(gsm1k_data_path)
gsm1k_df

,System,Date,GSM8k,GSM1k,Training compute (FLOP),Speculative Compute,Active Parameters,Open/Closed
0,claude-2.1,2023-07-11,0.887,0.894,3.800000e+24,NaN,NaN,Closed
1,claude-3-haiku-20240307,2024-03-04,0.785,0.785,NaN,NaN,NaN,Closed
2,claude-3-opus-20240229,2024-03-04,0.802,0.825,NaN,4.000000e+25,NaN,Closed
3,claude-3-sonnet-20240229,2024-03-04,0.719,0.744,NaN,NaN,NaN,Closed
4,codegemma-7b,2024-04-09,0.479,0.416,3.330000e+23,NaN,"7,000,000,000",Open
...,...,...,...,...,...,...,...,...
66,vicuna-33b-v1.3,2023-06-22,0.379,0.341,NaN,NaN,"33,000,000,000",Open
67,Xwin-Math-13B-V1.0,2024-03-07,0.631,0.529,NaN,NaN,"13,000,000,000",Open
68,Xwin-Math-7B-V1.0,2024-03-07,0.529,0.428,NaN,NaN,"7,000,000,000",Open
69,Yi-34B-Chat,2023-11-02,0.641,0.569,6.100000e+23,NaN,"34,000,000,000",Open


In [9]:
# Concatenate dfs
df = pd.concat([gsm1k_df, df], axis=0, join='outer', ignore_index=True)
df

,System,Date,GSM8k,GSM1k,Training compute (FLOP),Speculative Compute,Active Parameters,Open/Closed,Model size (parameters),Dataset size,Training compute notes,BBH,GPQA,MMLU,HELM MMLU,SEAL Coding,SEAL Instruction Following,SEAL Math,LMSys Elo,LMSys Elo Notes,LMSys Elo 95% CI,BBH Notes,GPQA Notes,MMLU Notes,HELM MMLU Notes,Trust in benchmark results,Trust notes
0,claude-2.1,2023-07-11,0.887,0.894,3.800000e+24,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,claude-3-haiku-20240307,2024-03-04,0.785,0.785,NaN,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,claude-3-opus-20240229,2024-03-04,0.802,0.825,NaN,4.000000e+25,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,claude-3-sonnet-20240229,2024-03-04,0.719,0.744,NaN,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,codegemma-7b,2024-04-09,0.479,0.416,3.330000e+23,NaN,"7,000,000,000",Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,XVerse-7B,2023-09-26,NaN,NaN,NaN,NaN,"7,000,000,000",Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
199,Yi-1.5-34B,2024-05-10,NaN,NaN,NaN,NaN,NaN,Open,NaN,NaN,NaN,NaN,0.060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Epoch evaluation,NaN,NaN,0.0,NaN
200,Yi-34B,2023-11-02,NaN,NaN,6.120000e+23,NaN,"34,000,000,000",Open,3.400000e+10,3.000000e+12,NaN,0.543,0.165,0.7635,NaN,NaN,NaN,NaN,1111.0,chat,NaN,NaN,Epoch evaluation,NaN,NaN,-1.0,MMLU-GPQA performance difference is relatively...
201,Yi-6B,2023-11-02,NaN,NaN,1.080000e+23,NaN,"6,000,000,000",Open,6.000000e+09,3.000000e+12,NaN,0.428,NaN,0.6385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [10]:
df.loc[df['System'] == 'Random chance', 'GSM1k'] = 0.0

In [11]:
# Ablate Llamas
if exclude_big_llama:
    df = df[~((df['System'].str.contains('Llama', case=False)) & (df['System'].str.contains('405')))]
if exclude_all_llamas:
    df = df[~df['System'].str.contains('Llama', case=False)]

In [12]:
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df

,System,Date,GSM8k,GSM1k,Training compute (FLOP),Speculative Compute,Active Parameters,Open/Closed,Model size (parameters),Dataset size,Training compute notes,BBH,GPQA,MMLU,HELM MMLU,SEAL Coding,SEAL Instruction Following,SEAL Math,LMSys Elo,LMSys Elo Notes,LMSys Elo 95% CI,BBH Notes,GPQA Notes,MMLU Notes,HELM MMLU Notes,Trust in benchmark results,Trust notes
0,claude-2.1,2023-07-11,0.887,0.894,3.800000e+24,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,claude-3-haiku-20240307,2024-03-04,0.785,0.785,NaN,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,claude-3-opus-20240229,2024-03-04,0.802,0.825,NaN,4.000000e+25,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,claude-3-sonnet-20240229,2024-03-04,0.719,0.744,NaN,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,codegemma-7b,2024-04-09,0.479,0.416,3.330000e+23,NaN,"7,000,000,000",Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,XVerse-7B,2023-09-26,NaN,NaN,NaN,NaN,"7,000,000,000",Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
199,Yi-1.5-34B,2024-05-10,NaN,NaN,NaN,NaN,NaN,Open,NaN,NaN,NaN,NaN,0.060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Epoch evaluation,NaN,NaN,0.0,NaN
200,Yi-34B,2023-11-02,NaN,NaN,6.120000e+23,NaN,"34,000,000,000",Open,3.400000e+10,3.000000e+12,NaN,0.543,0.165,0.7635,NaN,NaN,NaN,NaN,1111.0,chat,NaN,NaN,Epoch evaluation,NaN,NaN,-1.0,MMLU-GPQA performance difference is relatively...
201,Yi-6B,2023-11-02,NaN,NaN,1.080000e+23,NaN,"6,000,000,000",Open,6.000000e+09,3.000000e+12,NaN,0.428,NaN,0.6385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [13]:
# Filter out finetuned systems

finetuned_systems = [
 'Layer Normalization: Handwriting sequence generation',
 'ULM-FiT',
 'ADP-FAIRSEQ + NGRAMRES',
 'Cross-lingual alignment',
 'UnifiedQA',
 '$\\infty$-former (SM)',
 'FLAN 137B',
 'AlphaFold-Multimer',
 'Masked Autoencoders',
 'Contriever',
 'BERT-RBP',
 'Minerva',
 'BlenderBot 3',
 'PaLM-SayCan',
 'NMST+GPT-2',
 'Decaying Fast Weights Transformer (WT-103)',
 'GPT-2 + Progressive LRD',
 'U-PaLM',
 'Flan-T5 11B',
 'Flan-PaLM 540B',
 'Taiyi-Stable Diffusion',
 'OPT-IML (175B)',
 'SparseOPT-175B',
 'DiT-XL/2',
 'VideoMAE V2',
 'Segment Anything Model',
 'gLM',
 'MOSS-Moon-003',
 'WizardLM-7B',
 'InstructBLIP',
 'Guanaco-65B',
 'WizardCoder-15.5B',
 'Code Llama-34B',
 'Code Llama-7B',
 'TigerBot-70B',
 'MiniGPT4 (Vicuna finetune)',
 'LLaMA-7B (protein-oriented instructions finetuned)',
 'FinGPT-13B',
 'LLaVA 1.5',
 'CogVLM',
 'Volcano 13B',
 'SPHINX (Llama 2 13B)',
 'Orca 2-13B',
 'Llama Guard',
 'FunSearch',
 'Elyza',
 'Code Llama-70B',
 'Swallow',
 'Hermes-2 Theta Llama 3 70B',
]

df = df[~df['System'].isin(finetuned_systems)]
df = df[~df['System'].str.contains('Flan', case=False)]

## Merge SEAL Math with GSM1k

In [14]:
for i, row in df.iterrows():
  if pd.notna(row['SEAL Math']):
    df.at[i, 'GSM1k'] = row['SEAL Math']
df

,System,Date,GSM8k,GSM1k,Training compute (FLOP),Speculative Compute,Active Parameters,Open/Closed,Model size (parameters),Dataset size,Training compute notes,BBH,GPQA,MMLU,HELM MMLU,SEAL Coding,SEAL Instruction Following,SEAL Math,LMSys Elo,LMSys Elo Notes,LMSys Elo 95% CI,BBH Notes,GPQA Notes,MMLU Notes,HELM MMLU Notes,Trust in benchmark results,Trust notes
0,claude-2.1,2023-07-11,0.887,0.894,3.800000e+24,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,claude-3-haiku-20240307,2024-03-04,0.785,0.785,NaN,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,claude-3-opus-20240229,2024-03-04,0.802,0.825,NaN,4.000000e+25,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,claude-3-sonnet-20240229,2024-03-04,0.719,0.744,NaN,NaN,NaN,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,codegemma-7b,2024-04-09,0.479,0.416,3.330000e+23,NaN,"7,000,000,000",Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,XVerse-7B,2023-09-26,NaN,NaN,NaN,NaN,"7,000,000,000",Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
199,Yi-1.5-34B,2024-05-10,NaN,NaN,NaN,NaN,NaN,Open,NaN,NaN,NaN,NaN,0.060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Epoch evaluation,NaN,NaN,0.0,NaN
200,Yi-34B,2023-11-02,NaN,NaN,6.120000e+23,NaN,"34,000,000,000",Open,3.400000e+10,3.000000e+12,NaN,0.543,0.165,0.7635,NaN,NaN,NaN,NaN,1111.0,chat,NaN,NaN,Epoch evaluation,NaN,NaN,-1.0,MMLU-GPQA performance difference is relatively...
201,Yi-6B,2023-11-02,NaN,NaN,1.080000e+23,NaN,"6,000,000,000",Open,6.000000e+09,3.000000e+12,NaN,0.428,NaN,0.6385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


## Benchmarks vs Active Parameters

In [15]:
def convert_int(x):
    if pd.isna(x):
        return np.nan
    try:
        return int(str(x).replace(',', ''))
    except ValueError:
        return np.nan

df['Active Parameters'] = df['Active Parameters'].apply(convert_int)

In [16]:
color_map = {'Open': 'blue', 'Closed': 'darkorange'}

highlighted_models = ['Llama 3.1 405B', 'GPT-4 (original)', 'Qwen2-72B', 'Mistral Large 2', 
                      'Nemotron-4-340B Base', 'PaLM-2', 'Gemma 2 9B', 'LLaMa-2 7B', 'Gemma 1 2B', 'T5-Base', 'Chinchilla 70B']

# Create the plot
fig = px.scatter(df, x='Active Parameters', y='MMLU', 
                 color='Open/Closed',
                 title='MMLU vs Active Parameters',
                 hover_data=['System', 'MMLU', 'Active Parameters'],
                 color_discrete_map=color_map
                )
fig.update_layout(xaxis_type="log")

for i, row in df.iterrows():
    if row['System'] in highlighted_models:
        if row['System'] in ['GPT-4 (original)', 'Qwen2-72B', 'Gemma 2 9B', 'T5-Base']:
            ay = -20
        elif row['System'] == 'Llama 3.1 405B':
            ay = 20
        else:
            ay = 30

        ax = 20 if row['System'] in ['Llama 3.1 405B', 'Chinchilla 70B', 'Nemotron-4-340B Base'] else -10
        
        fig.add_annotation(
            x=np.log10(row['Active Parameters']),
            y=row['MMLU'],
            text=row["System"],
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1.5,
            visible=True,
            ay=ay,
            ax=ax
        )

# Show the plot
fig.show()

In [17]:
color_map = {'Open': 'blue', 'Closed': 'darkorange'}

highlighted_models = ['Llama 3.1 405B', 'Mistral Large 2', 'Llama 3.1 70B', 'Llama 3 70B']

# Create the plot
fig = px.scatter(df, x='Active Parameters', y='GPQA', 
                 color='Open/Closed',
                 title='GPQA vs Active Parameters',
                 hover_data=['System', 'GPQA', 'Active Parameters'],
                 color_discrete_map=color_map
                )

for i, row in df.iterrows():
    if row['System'] in highlighted_models:
        fig.add_annotation(
            x=np.log10(row['Active Parameters']),
            y=row['GPQA'],
            text=row["System"],
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1,
            visible=True,
            ay=-30,
            ax=-20
        )
fig.update_layout(xaxis_type="log")

# Show the plot
fig.show()

In [18]:
color_map = {'Open': 'blue', 'Closed': 'darkorange'}

highlighted_models = ['Nemotron-4-340B Base','DeepSeek-Coder-V2','Qwen2-72B', 'Mixtral8x22B', 'Gemma 2 27B', 
                      'PaLM-2', 'text-davinci-003']

# Create the plot
fig = px.scatter(df, x='Active Parameters', y='BBH', 
                 color='Open/Closed',
                 title='BBH vs Active Parameters',
                 hover_data=['System', 'BBH', 'Active Parameters'],
                 color_discrete_map=color_map
                )
fig.update_layout(xaxis_type="log")
for i, row in df.iterrows():
    if row['System'] in highlighted_models:
        ay = -40 if row['System'] == 'Nemotron-4-340B Base' else -30
        fig.add_annotation(
            x=np.log10(row['Active Parameters']),
            y=row['BBH'],
            text=row["System"],
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1,
            visible=True,
            ay=ay,
            ax=-30
        )

# Show the plot
fig.show()

In [19]:
color_map = {'Open': 'blue', 'Closed': 'darkorange'}

highlighted_models = ['Llama 3.1 405B', 'gpt-4', 'Llama 3 70B', 'Mixtral-8x22B-instruct-v0.1'] 
fig = px.scatter(df, x='Active Parameters', y='GSM1k', 
                 color='Open/Closed',
                 title='GSM1k vs Active Parameters',
                 hover_data=['System', 'GSM1k', 'Active Parameters'],
                 color_discrete_map=color_map
                )
fig.update_layout(xaxis_type="log")
for i, row in df.iterrows():
    if row['System'] in highlighted_models:
        fig.add_annotation(
            x=np.log10(row['Active Parameters']),
            y=row['GSM1k'],
            text=row["System"],
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1,
            visible=True,
            ay=-30,
            ax=-20
        )

# Show the plot
fig.show()

# Bench-to-bench comparisons for vetting purposes

In [20]:
bench1 = 'MMLU'
bench2 = 'GPQA' 
df[f'{bench1} (log-ratio)'] = -np.log(1/df[bench1] - 1)
df[f'{bench2} (log-ratio)'] = -np.log(1/df[bench2] - 1)

In [21]:
color_map = {'Open': 'blue', 'Closed': 'darkorange'}

# Create the plot
fig = px.scatter(df, x=f'{bench1} (log-ratio)', y=f'{bench2} (log-ratio)', color='Open/Closed',
                 title=f'{bench1} vs. {bench2} for Open and Closed Models',
                 labels={f'{bench1} (log-ratio)': f'{bench1} Score', f'{bench2} (log-ratio)': f'{bench2} Score', 'Date': 'Date', 'System': 'Model'},
                 hover_data=['System', 'Date', f'{bench1}', f'{bench2}'],
                 color_discrete_map=color_map)

# Improve the layout
fig.update_layout(
    width=800,
    height=400,
    xaxis_title=f"{bench1} (log-ratio)",
    yaxis_title=f"{bench2} (log-ratio)",
    legend_title="Model accessibility",
    font=dict(size=12),
    hovermode="closest"
)

# Show the plot
fig.show()

In [22]:
mmlu_gqpa_df = df.dropna(subset=[f'{bench1} (log-ratio)', f'{bench2} (log-ratio)'])
largest_diffs = (mmlu_gqpa_df[f'{bench1} (log-ratio)'] - mmlu_gqpa_df[f'{bench2} (log-ratio)']).sort_values(ascending=False).index
for idx in largest_diffs:
    print(df.loc[idx, 'System'], df.loc[idx, f'{bench1} (log-ratio)'] - df.loc[idx, f'{bench2} (log-ratio)'])

Yi-34B 2.793451167676974
DeepSeek-67B 2.2289073920244826
gpt-4-0613 2.1868672248322296
Mistral 7B 2.18400124416153
Qwen2-72B 2.171230878334952
Llama 3 70B 1.9847264228868222
Claude 3 Opus 1.9674392364164304
GPT-4o 1.9587644946030616
Mixtral 8x22B 1.9397056511368995
GPT-4o mini 1.9093280408140076
Claude 2 1.9009239272223497
Llama 3.1 405B 1.8917445806250093
LLaMa-2 70B 1.8887344032198596
DBRX-Instruct 1.85875581893977
Claude 3.5 Sonnet 1.8111742240799946
Llama 3.1 70B 1.8092505609727965
Claude 3 Sonnet 1.7680361224681302
GPT-3.5-turbo-16k 1.7524152916632334
Gemini 1.5 Pro (April 2024) 1.7223864594960092
Gemma 2 27B 1.7064401051052145
Gemini 1.5 Flash Preview 1.7035825258088986
Mistral Large 2 1.6982334112172317
Gemma 2 9B 1.6454487649053096
Llama 3 8B 1.633840457043604
Nemotron-4-340B Instruct 1.6215046373443438
Gemini 1.0 Pro 1.5800816296808011
Random chance 0.0


# LMSys leaderboard

In [23]:
# From 
lmsys_leaderboard_bootstrap_elo_lu = pd.read_csv('https://docs.google.com/spreadsheets/d/12zpanuQ1Vf_ZsZ6yjIUwsN7uGPBv3ChLnEOH-g9yZDA/export?format=csv')
lmsys_leaderboard_bootstrap_elo_lu = lmsys_leaderboard_bootstrap_elo_lu.iloc[:, 1:]  # remove first column which is meaningless
lmsys_leaderboard_bootstrap_elo_lu

,gpt-4o-2024-05-13,gpt-4o-mini-2024-07-18,claude-3-5-sonnet-20240620,gemini-advanced-0514,llama-3.1-405b-instruct,gemini-1.5-pro-api-0514,gemini-1.5-pro-api-0409-preview,gpt-4-turbo-2024-04-09,gpt-4-1106-preview,claude-3-opus-20240229,athene-70b-0725,gpt-4-0125-preview,llama-3.1-70b-instruct,yi-large-preview,gemini-1.5-flash-api-0514,gemma-2-27b-it,yi-large,nemotron-4-340b-instruct,bard-jan-24-gemini-pro,glm-4-0520,llama-3-70b-instruct,claude-3-sonnet-20240229,reka-core-20240501,command-r-plus,gemma-2-9b-it,qwen2-72b-instruct,gpt-4-0314,glm-4-0116,qwen-max-0428,claude-3-haiku-20240307,deepseek-coder-v2,llama-3.1-8b-instruct,reka-flash-preview-20240611,gpt-4-0613,qwen1.5-110b-chat,yi-1.5-34b-chat,mistral-large-2402,reka-flash-21b-20240226-online,llama-3-8b-instruct,claude-1,command-r,mistral-medium,reka-flash-21b-20240226,qwen1.5-72b-chat,mixtral-8x22b-instruct-v0.1,claude-2.0,gemini-pro-dev-api,zephyr-orpo-141b-A35b-v0.1,qwen1.5-32b-chat,mistral-next,phi-3-medium-4k-instruct,starling-lm-7b-beta,claude-2.1,gpt-3.5-turbo-0613,mixtral-8x7b-instruct-v0.1,claude-instant-1,yi-34b-chat,gemini-pro,qwen1.5-14b-chat,gpt-3.5-turbo-0314,wizardlm-70b,gpt-3.5-turbo-0125,dbrx-instruct-preview,phi-3-small-8k-instruct,tulu-2-dpo-70b,llama-2-70b-chat,openchat-3.5-0106,vicuna-33b,snowflake-arctic-instruct,starling-lm-7b-alpha,gemma-1.1-7b-it,nous-hermes-2-mixtral-8x7b-dpo,llama2-70b-steerlm-chat,pplx-70b-online,deepseek-llm-67b-chat,openchat-3.5,openhermes-2.5-mistral-7b,mistral-7b-instruct-v0.2,qwen1.5-7b-chat,gpt-3.5-turbo-1106,phi-3-mini-4k-instruct,llama-2-13b-chat,dolphin-2.2.1-mistral-7b,solar-10.7b-instruct-v1.0,phi-3-mini-4k-instruct-june-2024,wizardlm-13b,zephyr-7b-beta,mpt-30b-chat,pplx-7b-online,zephyr-7b-alpha,codellama-34b-instruct,vicuna-13b,codellama-70b-instruct,gemma-7b-it,llama-2-7b-chat,phi-3-mini-128k-instruct,qwen-14b-chat,falcon-180b-chat,guanaco-33b,gemma-1.1-2b-it,stripedhyena-nous-7b,olmo-7b-instruct,mistral-7b-instruct,vicuna-7b,palm-2,gemma-2b-it,qwen1.5-4b-chat,koala-13b,chatglm3-6b,gpt4all-13b-snoozy,mpt-7b-chat,chatglm2-6b,RWKV-4-Raven-14B,alpaca-13b,oasst-pythia-12b,chatglm-6b,fastchat-t5-3b,stablelm-tuned-alpha-7b,dolly-v2-12b,llama-13b
0,1286.563455,1279.608815,1274.564379,1266.819548,1263.930629,1262.800113,1256.751200,1259.245346,1251.302638,1246.760897,1246.975827,1245.403904,1231.261502,1240.579372,1227.633631,1217.333304,1214.194600,1213.808952,1207.248663,1209.357038,1207.080257,1202.432095,1201.256188,1191.611806,1190.367829,1185.927943,1186.314345,1185.426038,1181.411865,1177.081150,1183.789524,1161.297640,1165.446680,1162.079717,1162.169726,1155.032525,1157.033520,1154.975771,1152.449178,1147.384927,1150.788865,1147.545597,1148.235700,1146.857984,1145.494533,1134.056454,1130.061195,1124.566277,1125.848089,1122.868939,1125.179143,1120.464656,1118.266832,1117.700619,1114,1110.580958,1113.647393,1110.270015,1107.595299,1108.959096,1104.118028,1106.121387,1101.942314,1104.556936,1103.714793,1095.159053,1094.081357,1089.516859,1091.639150,1091.191383,1085.533770,1075.749723,1078.874043,1078.454370,1074.880985,1073.584866,1073.645051,1073.257736,1068.678238,1064.406846,1067.189825,1061.298298,1057.464642,1061.934840,1061.155330,1058.210607,1055.467933,1047.889266,1041.020906,1038.351736,1040.433205,1042.904689,1032.235059,1035.848555,1036.705633,1039.397830,1033.082207,1028.477258,1026.970100,1024.312538,1018.190363,1016.452466,1011.145134,1001.019073,1000.320046,995.256003,988.437218,958.395166,961.292969,935.567166,924.081608,930.543839,918.995447,901.694532,895.192400,883.743569,867.977489,837.073711,818.981457,808.397845
1,1288.428940,1287.494945,1274.828503,1268.867005,1263.724865,1263.094929,1258.320223,1258.824223,1252.959457,1250.215882,1255.401278,1247.008432,1243.378408,1241.854920,1231.290454,1220.257022,1217.498630,1213.345389,1208.149339,1209.489786,1209.536015,1203.785052,1203.180498,1191.145427,1189.836713,1189.163734,1189.473124,1184.030035,1186.551048,1181.552980,1180.795775,1170.386280,1168.0

In [24]:
# Alphabetical order
lmsys_leaderboard_bootstrap_elo_lu.mean().round().sort_index()

RWKV-4-Raven-14B               922.0
alpaca-13b                     902.0
athene-70b-0725               1246.0
bard-jan-24-gemini-pro        1208.0
chatglm-6b                     880.0
                               ...  
yi-large                      1212.0
yi-large-preview              1240.0
zephyr-7b-alpha               1042.0
zephyr-7b-beta                1053.0
zephyr-orpo-141b-A35b-v0.1    1126.0
Length: 120, dtype: float64

In [25]:
# Descending order of Elo
lmsys_leaderboard_bootstrap_elo_lu.mean().round().sort_values(ascending=False)

gpt-4o-2024-05-13             1286.0
gpt-4o-mini-2024-07-18        1280.0
claude-3-5-sonnet-20240620    1271.0
gemini-advanced-0514          1266.0
llama-3.1-405b-instruct       1262.0
                               ...  
chatglm-6b                     880.0
fastchat-t5-3b                 869.0
stablelm-tuned-alpha-7b        840.0
dolly-v2-12b                   823.0
llama-13b                      799.0
Length: 120, dtype: float64

## Trust in evaluations

In [26]:
# Models whose evaluations we have a concrete reason to distrust
list(df[df['Trust in benchmark results'] < 0]['System'])

['DBRX-Instruct',
 'DeepSeek-67B',
 'Falcon 180B',
 'Gemma 2 27B',
 'gpt-4-0125-preview',
 'gpt-4-0613',
 'Llama 3 8B',
 'Mistral 7B',
 'Mixtral 8x22B',
 'Qwen2-72B',
 'Reka Core',
 'Yi-34B']

In [27]:
# Models whose evaluations we have a concrete reason to trust
list(df[df['Trust in benchmark results'] > 0]['System'])

['Claude 3 Opus',
 'Claude 3 Sonnet',
 'Gemini 1.0 Pro',
 'Gemini 1.5 Pro (April 2024)',
 'GPT-3.5-turbo-16k',
 'GPT-4 (original)',
 'gpt-4-turbo-2024-04-09',
 'Llama 3 70B',
 'Mistral Large']

# Analysis

In [28]:
df['Before November 2023'] = ['Before' if pd.to_datetime(row['Date']) < pd.to_datetime('2023-11-01') else 'After' for i, row in df.iterrows()]

## Date

In [29]:
filtered_dfs = {}

for i, bench in enumerate(benchmarks_to_analyze):
  print(bench)
  filtered_df = df[~(df['System'] == 'Random chance')]
  if old_models_only:
    filtered_df = filtered_df[filtered_df['Before November 2023'] == 'Before']
  elif new_models_only:
    filtered_df = filtered_df[filtered_df['Before November 2023'] == 'After']
  if non_suspects_only:
    if bench == 'GPQA':
      # GPQA was released November 20, 2023
      old_df = filtered_df[filtered_df['Before November 2023'] == 'Before']
      new_df = filtered_df[filtered_df['Before November 2023'] == 'After']
      new_df = new_df[new_df['Trust in benchmark results'] >= 0]
      filtered_df = pd.concat([old_df, new_df])
    elif bench == 'MMLU':
      filtered_df = filtered_df[filtered_df['Trust in benchmark results'] >= 0]
  elif trusted_only:
    filtered_df = filtered_df[filtered_df['Trust in benchmark results'] > 0]

  # Find the max score over time for each category
  filtered_df = filtered_df.sort_values('Date')
  filtered_dfs[bench] = filtered_df

MMLU
GPQA
GSM1k
BBH


In [30]:
max_rows_by_bench = defaultdict(dict)
for i, bench in enumerate(benchmarks_to_analyze):
    print(bench)
    filtered_df = filtered_dfs[bench]

    # Find the max score over time for each category
    filtered_df = filtered_df.sort_values('Date')
    max_rows = defaultdict(list)
    for category in ['Open', 'Closed']:
        category_df = filtered_df[filtered_df['Open/Closed'] == category].dropna(subset=[bench])
        max_score = 0
        for i, row in category_df.iterrows():
            score = -np.log(1 - row[bench])
            if score > max_score:
                # Models released on the same date: only include the maximum of those models
                # Example: Claude 3 Sonnet vs. Opus
                max_score = score
                max_row = {'Date': row['Date'], 'System': row['System'], 'Score': score}
                if len(max_rows[category]) > 0 and max_rows[category][-1]['Date'] == row['Date']:
                    max_rows[category][-1] = max_row
                else:
                    max_rows[category].append(max_row)

        print(f"{category} models:")
        for row in max_rows[category]:
            print(row['System'], row['Date'], row['Score'])
        print()

    max_rows_by_bench[bench] = max_rows

MMLU
Open models:
T5-Small 2019-10-23 00:00:00 0.3106095770954856
GPT-NeoX 20B 2022-02-09 00:00:00 0.4094731295057033
OPT-66B 2022-05-02 00:00:00 0.44613086483417935
BLOOM-176B 2022-07-11 00:00:00 0.49642974348225827
GLM 130B 2022-08-04 00:00:00 0.5942072327050416
LLaMa-1 65B 2023-02-24 00:00:00 1.0051219455807707
LLaMa-2 70B 2023-07-18 00:00:00 1.1679623668029027
Grok-1 2023-11-03 00:00:00 1.3093333199837622
Camelidae-8x34B 2024-01-05 00:00:00 1.410587053688935
Cohere Command R+ 2024-04-04 00:00:00 1.4146938356415886
Llama 3 70B 2024-04-18 00:00:00 1.7147984280919264
Llama 3.1 405B 2024-07-23 00:00:00 2.0635681925235456

Closed models:
text-davinci-001 2020-05-28 00:00:00 0.5058380822549516
Gopher 280B 2021-12-08 00:00:00 0.916290731874155
code-davinci-002 2022-03-01 00:00:00 1.1457038962019603
PaLM 540B 2022-04-04 00:00:00 1.2482730632225159
GPT-4 (original) 2023-03-15 00:00:00 1.995100393246085
Claude 3 Opus 2024-03-04 00:00:00 2.0249533563957662
GPT-4o 2024-05-13 00:00:00 2.0557250

In [31]:
max_rows_by_bench

defaultdict(dict,
            {'MMLU': defaultdict(list,
                         {'Open': [{'Date': Timestamp('2019-10-23 00:00:00'),
                            'System': 'T5-Small',
                            'Score': 0.3106095770954856},
                           {'Date': Timestamp('2022-02-09 00:00:00'),
                            'System': 'GPT-NeoX 20B',
                            'Score': 0.4094731295057033},
                           {'Date': Timestamp('2022-05-02 00:00:00'),
                            'System': 'OPT-66B',
                            'Score': 0.44613086483417935},
                           {'Date': Timestamp('2022-07-11 00:00:00'),
                            'System': 'BLOOM-176B',
                            'Score': 0.49642974348225827},
                           {'Date': Timestamp('2022-08-04 00:00:00'),
                            'System': 'GLM 130B',
                            'Score': 0.5942072327050416},
                           {'Date': Ti

### Estimate lag by area between curves

In [32]:
max_dfs_for_auc = defaultdict(dict)
for i, bench in enumerate(benchmarks_to_analyze):
    print(bench)
    max_rows = max_rows_by_bench[bench]

    if max_rows['Open'][-1]['Score'] < max_rows['Closed'][-1]['Score']:
        lead_category = 'Closed'
        lag_category = 'Open'
    else:
        lead_category = 'Open'
        lag_category = 'Closed'

    lead_rows = [row for row in max_rows[lead_category]]
    lag_rows = [row for row in max_rows[lag_category]]
    # Start scores from the random chance baseline
    lead_rows.insert(0, {'Date': lead_rows[0]['Date'], 'System': 'Random chance', 'Score': df[df['System'] == 'Random chance'][bench].values[0]})
    lag_rows.insert(0, {'Date': lag_rows[0]['Date'], 'System': 'Random chance', 'Score': df[df['System'] == 'Random chance'][bench].values[0]})
    # We don't know how long it will take the lag category to catch up from now
    # So truncate the lag category to the last score that surpassed the lead category,
    # and truncate the lead category to that score too
    lag_row_to_truncate_to = None
    for lead_row in lead_rows[::-1]:
        if lag_row_to_truncate_to is not None:
            break
        if lead_row['Score'] < lag_rows[-1]['Score']:
            for i, lag_row in enumerate(lag_rows):
                if lag_row['Score'] > lead_row['Score']:
                    lag_rows = lag_rows[:i+1]
                    lag_row_to_truncate_to = lag_row
                    break

    next_lead_row = [row for row in lead_rows if row['Score'] > lag_row_to_truncate_to['Score']][0]
    lead_rows = [row for row in lead_rows if row['Score'] <= lag_row_to_truncate_to['Score']]
    lead_rows.append({'Date': lead_rows[-1]['Date'], 'System': f'{next_lead_row["System"]} (Truncated)', 'Score': lag_row_to_truncate_to['Score']})

    max_dfs_for_auc[bench][lead_category] = pd.DataFrame(lead_rows)
    max_dfs_for_auc[bench][lag_category] = pd.DataFrame(lag_rows)


MMLU
GPQA
GSM1k
BBH


In [33]:
max_dfs_for_auc['MMLU']['Closed']

,Date,System,Score
0,2020-05-28,Random chance,0.250000
1,2020-05-28,text-davinci-001,0.505838
2,2021-12-08,Gopher 280B,0.916291
3,2022-03-01,code-davinci-002,1.145704
4,2022-04-04,PaLM 540B,1.248273
5,2023-03-15,GPT-4 (original),1.995100
6,2024-03-04,Claude 3 Opus,2.024953
7,2024-05-13,GPT-4o,2.055725
8,2024-05-13,Claude 3.5 Sonnet (Truncated),2.063568


In [34]:
# Plot time on y axis and score on x axis
bench = 'GPQA'
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=max_dfs_for_auc[bench]['Closed']['Score'],
        y=max_dfs_for_auc[bench]['Closed']['Date'], 
        mode='lines+markers',
        marker=dict(color='orange'),
        text=max_dfs_for_auc[bench]['Closed']['System'],
        textposition='bottom center',
        name='Closed',
        line=dict(shape='vh')
    )
)
fig.add_trace(
    go.Scatter(
        x=max_dfs_for_auc[bench]['Open']['Score'],
        y=max_dfs_for_auc[bench]['Open']['Date'],
        mode='lines+markers',
        marker=dict(color='blue'),
        text=max_dfs_for_auc[bench]['Open']['System'],
        textposition='bottom center',
        name='Open',
        line=dict(shape='vh')
    )
)
fig.update_layout(
    width=650,
    height=600,
    title=f"{bench} curves for AUC calculation",
    xaxis_title="Accuracy",
    yaxis_title="Date",
)
fig.show()

In [35]:
max_dfs_for_auc[bench][category]

,Date,System,Score
0,2023-06-13,Random chance,0.250000
1,2023-06-13,gpt-4-0613,0.397497
2,2023-07-11,Claude 2,0.435409
3,2023-11-06,gpt-4-1106-preview,0.553385
4,2024-03-04,Claude 3 Opus,0.652005
5,2024-05-13,GPT-4o,0.673345
6,2024-05-13,Claude 3.5 Sonnet (Truncated),0.711311


In [36]:
# We estimate the mean date lag using integrals
# First, calculate the area under each date(performance) function
integrals = defaultdict(lambda: defaultdict(list))
auc_lags = {}
for bench in benchmarks_to_analyze:
    integrals[bench] = {}
    for category in ['Open', 'Closed']:
        integral = 0
        max_df = max_dfs_for_auc[bench][category]
        for i, row in max_df.iterrows():
            if i == 0:
                continue
            # Measure date in months since epoch
            date_diff = (row['Date'] - datetime(1970, 1, 1)).days/30
            score_diff = row['Score'] - max_df.iloc[i-1]['Score']
            integral += score_diff * date_diff
        integrals[bench][category] = integral
    # Now take the difference between the two integrals and divide by the score range
    # We made the score range the same for each category, so use 'Open' arbitrarily
    score_range = max_dfs_for_auc[bench]['Open'].iloc[-1]['Score'] - max_dfs_for_auc[bench]['Open'].iloc[0]['Score']
    print(f"Score range for {bench}: {score_range:.2f}")
    mean_lag = (integrals[bench]['Open'] - integrals[bench]['Closed']) / score_range
    auc_lags[bench] = mean_lag
    print(f"Lag for {bench}: {mean_lag:.0f} months")

Score range for MMLU: 1.81
Lag for MMLU: 14 months
Score range for GPQA: 0.46
Lag for GPQA: 3 months
Score range for GSM1k: 3.02
Lag for GSM1k: 5 months
Score range for BBH: 0.83
Lag for BBH: 6 months


### [MAIN RESULT] Estimate lag by discrete catch-up times

In [37]:
discrete_lags = defaultdict(lambda: defaultdict(list))
for i, bench in enumerate(benchmarks_to_analyze):
  print(bench)
  max_rows = max_rows_by_bench[bench]

  already_matched = set()
  open_winner_score = 0
  for closed_row in max_rows['Closed']:
    if closed_row['Score'] <= open_winner_score + benchmark_noise_tolerance:
      # Skip if there has already been a better open model
      continue
    for open_row in max_rows['Open']:
      if open_row['Score'] > (closed_row['Score'] - benchmark_noise_tolerance) and open_row['Date'] not in already_matched:
        open_winner_score = open_row['Score']
        lag_months = (open_row['Date'] - closed_row['Date']).days/365*12
        closed_acc = 100 * (1 - np.exp(-closed_row['Score']))
        open_acc = 100 * (1 - np.exp(-open_row['Score']))
        print(f"{open_row['System']} at {open_acc:.2f}% matched or exceeded " + 
              f"{closed_row['System']} at {closed_acc:.2f}% after " + 
              f"{lag_months:.1f} months")
        # already_matched.add(open_row['Date'])
        discrete_lags[bench]['Closed system'].append(closed_row['System'])
        discrete_lags[bench]['Closed performance level'].append(closed_acc)
        discrete_lags[bench]['Closed date'].append(closed_row['Date'])
        discrete_lags[bench]['Open system'].append(open_row['System'])
        discrete_lags[bench]['Open performance level'].append(open_acc)
        discrete_lags[bench]['Open date'].append(open_row['Date'])
        discrete_lags[bench]['Lag (months)'].append(lag_months)

        break

  print(f"{bench} mean lag: {np.mean(discrete_lags[bench]['Lag (months)']):.1f} months\n")

discrete_lags

MMLU
BLOOM-176B at 39.13% matched or exceeded text-davinci-001 at 39.70% after 25.4 months
LLaMa-1 65B at 63.40% matched or exceeded Gopher 280B at 60.00% after 14.6 months
LLaMa-2 70B at 68.90% matched or exceeded code-davinci-002 at 68.20% after 16.6 months
Grok-1 at 73.00% matched or exceeded PaLM 540B at 71.30% after 19.0 months
Llama 3.1 405B at 87.30% matched or exceeded GPT-4 (original) at 86.40% after 16.3 months
MMLU mean lag: 18.4 months

GPQA
DeepSeek-Coder at 43.00% matched or exceeded gpt-4-0613 at 32.80% after 4.6 months
Llama 3.1 405B at 50.90% matched or exceeded Claude 3 Opus at 47.90% after 4.6 months
GPQA mean lag: 4.6 months

GSM1k
Mixtral-8x22B-instruct-v0.1 at 76.00% matched or exceeded gpt-3.5-turbo at 75.30% after 10.2 months
Llama 3 70B at 90.12% matched or exceeded claude-2.1 at 89.40% after 9.3 months
Llama 3.1 405B at 95.60% matched or exceeded gpt-4-0125-preview at 95.10% after 5.9 months
GSM1k mean lag: 8.4 months

BBH
GPT-NeoX 20B at 40.25% matched or exc

defaultdict(<function __main__.<lambda>()>,
            {'MMLU': defaultdict(list,
                         {'Closed system': ['text-davinci-001',
                           'Gopher 280B',
                           'code-davinci-002',
                           'PaLM 540B',
                           'GPT-4 (original)'],
                          'Closed performance level': [39.7,
                           60.0,
                           68.2,
                           71.3,
                           86.4],
                          'Closed date': [Timestamp('2020-05-28 00:00:00'),
                           Timestamp('2021-12-08 00:00:00'),
                           Timestamp('2022-03-01 00:00:00'),
                           Timestamp('2022-04-04 00:00:00'),
                           Timestamp('2023-03-15 00:00:00')],
                          'Open system': ['BLOOM-176B',
                           'LLaMa-1 65B',
                           'LLaMa-2 70B',
                     

In [38]:
all_lags = []
for bench in benchmarks_to_analyze:
    all_lags.extend(discrete_lags[bench]['Lag (months)'])
all_lags = np.array(all_lags)
print(f"Mean lag: {np.mean(all_lags):.0f} months")
print(f"Median lag: {np.median(all_lags):.0f} months")
print(f"95% of lags are less than {np.percentile(all_lags, 95):.0f} months")
print(f"5% of lags are less than {np.percentile(all_lags, 5):.0f} months")


Mean lag: 13 months
Median lag: 15 months
95% of lags are less than 22 months
5% of lags are less than 5 months


In [39]:
print(list(all_lags))

[25.446575342465753, 14.564383561643837, 16.56986301369863, 19.002739726027396, 16.306849315068494, 4.635616438356164, 4.635616438356164, 10.158904109589042, 9.271232876712329, 5.917808219178082, 20.449315068493153, 18.213698630136985, 5.58904109589041]


In [40]:
# Weighted average of compute lag and benchmark lags
compute_lag = 15  # Check this number is up to date with compute_analysis.ipynb
# Compute lag is weighted as much as the benchmark lags combined
weights = np.concatenate([np.ones(len(all_lags)), [compute_lag]])
all_lags_with_compute = np.concatenate([all_lags, [compute_lag]])
weighted_avg_lag = np.sum(all_lags_with_compute * weights) / np.sum(weights)
print(f"Weighted average lag: {weighted_avg_lag:.0f} months")

Weighted average lag: 14 months


In [41]:
# Create the figure with 1x4 subplots
fig = make_subplots(
    rows=1, 
    cols=4, 
    subplot_titles=benchmarks_to_analyze,
    horizontal_spacing=0.1
)

# Loop through each benchmark to plot the discrete_lags
for idx, bench in enumerate(benchmarks_to_analyze):
    for i in range(len(discrete_lags[bench]['Closed performance level'])):
        fig.add_trace(go.Scatter(
            y=[discrete_lags[bench]['Closed performance level'][i], discrete_lags[bench]['Closed performance level'][i]],
            x=[0, discrete_lags[bench]['Lag (months)'][i]],
            mode='lines',
            line=dict(color='rgb(230, 230, 245)', width=2),
            showlegend=False,
        ), row=1, col=idx + 1)
    
    # Plot markers for Closed and Open
    fig.add_trace(go.Scatter(
        x=np.zeros_like(discrete_lags[bench]['Closed performance level']),
        y=discrete_lags[bench]['Closed performance level'],
        mode='markers',
        marker=dict(color=color_map['Closed'], size=10),
        name='Closed (new state-of-the-art)',
        text=discrete_lags[bench]['Closed system'],
        showlegend=(idx == 0),
    ), row=1, col=idx + 1)
    fig.add_trace(go.Scatter(
        x=discrete_lags[bench]['Lag (months)'],
        y=discrete_lags[bench]['Closed performance level'],
        mode='markers',
        marker=dict(color=color_map['Open'], size=10),
        name='Open (similar or better performance)',
        text=discrete_lags[bench]['Open system'],
        showlegend=(idx == 0),
    ), row=1, col=idx + 1)

    if idx == 0:
        fig.update_yaxes(
            title='Accuracy level (%)',
            row=1, col=idx + 1
        )
    else:
        fig.update_yaxes(
            showticklabels=False,
            row=1, col=idx + 1
        )
    fig.update_yaxes(
        range=[0, 100],
        showgrid=False,
        row=1, col=idx + 1
    )

fig.update_xaxes(
    range=[-5, 32],
    title='Lag (months)',
    showgrid=False,
)

fig.update_layout(
    width=600,
    height=400,
    title=f'Open models have lagged on benchmarks by {np.percentile(all_lags, 5):.0f} to {np.percentile(all_lags, 95):.0f} months',
    template='plotly_white',
    # showlegend=False,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.2,
        xanchor='left',
        x=0,
    ),
    margin=dict(l=10, r=10, t=160, b=10),
)

if save:
    save_plot(fig, results_dir, f"benchmark_lags_{'_'.join(benchmarks_to_analyze)}_all")

# Show the plot
fig.show()

In [42]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=all_lags, nbinsx=20, opacity=0.75))
fig.update_layout(
    title='Distribution of lag (months)',
    xaxis_title='Lag (months)',
    yaxis_title='Count',
    template='plotly_white',
)
fig.show()


### Estimate lag by linear regression

In [43]:
fig = make_subplots(rows=2, cols=2, subplot_titles=benchmarks_to_analyze, vertical_spacing=0.2, horizontal_spacing=0.2)
for i, bench in enumerate(benchmarks_to_analyze):
  for category in ['Open', 'Closed']:
    category_df = pd.DataFrame(max_rows_by_bench[bench][category])

    fig.append_trace(
        go.Scatter(
            x=category_df['Date'],
            y=category_df['Score'],
            mode='markers',
            marker=dict(color=color_map[category]),
            text=category_df['System'],
            name=category,
            legendgroup=category,
            showlegend=True if i == 0 else False
        ),
        row=i//2 + 1, col=i%2 + 1
    )
fig.update_layout(
    width=600,
    height=400,
    margin=dict(l=10, r=10, t=70, b=10),
    font=dict(size=12),
    hovermode="closest",
)
fig.show()

In [44]:
reg_results = defaultdict(dict)
for i, bench in enumerate(benchmarks_to_analyze):
  for category in ['Open', 'Closed']:
    category_df = pd.DataFrame(max_rows_by_bench[bench][category])

    # Ignore the flatter part of the curve close to random chance
    random_chance_level = df[df['System'] == 'Random chance'][bench].values[0]
    thres = -np.log(1 - random_chance_level) + 0.2
    category_df = category_df[category_df['Score'] > thres]

    # Do regression
    category_df['year'] = datetime_to_float_year(category_df['Date'])
    results = fit_ols_regression(category_df, ['Score'], 'year')
    print(f"{bench} {category} R^2 of {results.rsquared:.2f}, equation: {results.params[0]:.2f} + {results.params[1]:.2f} * score")
    reg_results[bench][category] = results

MMLU Open R^2 of 0.94, equation: 2021.85 + 1.44 * score
MMLU Closed R^2 of 0.94, equation: 2019.56 + 2.19 * score
GPQA Open R^2 of 1.00, equation: 2021.09 + 4.87 * score
GPQA Closed R^2 of 0.78, equation: 2023.44 + 1.12 * score
GSM1k Open R^2 of 0.74, equation: 2023.49 + 0.40 * score
GSM1k Closed R^2 of 0.86, equation: 2022.74 + 0.44 * score
BBH Open R^2 of 0.76, equation: 2022.07 + 1.38 * score
BBH Closed R^2 of 0.85, equation: 2021.85 + 1.08 * score


/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_59619/1097503719.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_59619/1097503719.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_59619/1097503719.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/

In [45]:
# Find the average of the area between curves
for bench in benchmarks_to_analyze:
    open_slope = reg_results[bench]['Open'].params[1]
    open_intercept = reg_results[bench]['Open'].params[0]
    closed_slope = reg_results[bench]['Closed'].params[1]
    closed_intercept = reg_results[bench]['Closed'].params[0]

    random_chance_level = df[df['System'] == 'Random chance'][bench].values[0]
    min_score = -np.log(1 - random_chance_level)
    # print(f"Possible max scores: Open: {max_rows_by_bench[bench]['Open'][-1]['System']} at {max_rows_by_bench[bench]['Open'][-1]['Score']:.2f}, Closed: {max_rows_by_bench[bench]['Closed'][-1]['System']} at {max_rows_by_bench[bench]['Closed'][-1]['Score']:.2f}")
    max_score = min(max_rows_by_bench[bench]['Open'][-1]['Score'], max_rows_by_bench[bench]['Closed'][-1]['Score'])
    # print(f"Min score: {min_score:.2f}, max score: {max_score:.2f}")

    # Use the formula for the area between two lines
    avg_lag = 0.5 * (open_slope - closed_slope) * (min_score + max_score) + (open_intercept - closed_intercept)
    print(f"{bench}: {avg_lag*12:.0f} months")

MMLU: 17 months
GPQA: -6 months
GSM1k: 8 months
BBH: 7 months


/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_59619/122481273.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_59619/122481273.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_59619/122481273.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipyker

## Sample the lag at fixed performance levels

In [46]:
# performance_levels = [35, 45, 55, 65, 75, 85, 95]
# performance_levels = [30, 40, 50, 60, 70, 80, 90]
performance_levels = list(range(35, 100, 5))

In [47]:
rows_that_passed = defaultdict(lambda: defaultdict(dict))
time_deltas = defaultdict(lambda: defaultdict(list))
for bench in benchmarks_to_analyze:
    for category in ['Open', 'Closed']:
        row_idx = 0
        for pl in performance_levels:
            while row_idx < len(max_rows_by_bench[bench][category]):
                row = max_rows_by_bench[bench][category][row_idx]
                if row['Score'] >= -np.log(1 - pl/100):
                    rows_that_passed[bench][category][pl] = row
                    break
                row_idx += 1
    for pl in performance_levels:
        if pl in rows_that_passed[bench]['Closed'] and pl in rows_that_passed[bench]['Open']:
            time_deltas[bench]['Closed performance level'].append(pl)
            open_row = rows_that_passed[bench]['Open'][pl]
            closed_row = rows_that_passed[bench]['Closed'][pl]
            td = (open_row['Date'] - closed_row['Date']).days/365*12
            time_deltas[bench]['Lag (months)'].append(td)
            time_deltas[bench]['Open system'].append(open_row['System'])
            time_deltas[bench]['Closed system'].append(closed_row['System'])

time_deltas

defaultdict(<function __main__.<lambda>()>,
            {'MMLU': defaultdict(list,
                         {'Closed performance level': [35,
                           40,
                           45,
                           50,
                           55,
                           60,
                           65,
                           70,
                           75,
                           80,
                           85],
                          'Lag (months)': [23.145205479452056,
                           7.857534246575343,
                           14.564383561643837,
                           14.564383561643837,
                           14.564383561643837,
                           14.564383561643837,
                           16.56986301369863,
                           19.002739726027396,
                           9.731506849315068,
                           13.150684931506849,
                           16.306849315068494],
                

In [48]:
# Create the figure with 1x4 subplots
fig = make_subplots(
    rows=1, 
    cols=4, 
    shared_yaxes=True, 
    subplot_titles=benchmarks_to_analyze
)

# Loop through each benchmark to plot the time_deltas
for idx, bench in enumerate(benchmarks_to_analyze):
    for i in range(len(time_deltas[bench]['Closed performance level'])):
        fig.add_trace(go.Scatter(
            y=[time_deltas[bench]['Closed performance level'][i], time_deltas[bench]['Closed performance level'][i]],
            x=[0, time_deltas[bench]['Lag (months)'][i]],
            mode='lines',
            line=dict(color='rgb(230, 230, 245)', width=2),
            showlegend=False,
        ), row=1, col=idx + 1)
    
    # Plot markers for Closed and Open
    fig.add_trace(go.Scatter(
        y=time_deltas[bench]['Closed performance level'],
        x=np.zeros_like(time_deltas[bench]['Closed performance level']),
        mode='markers',
        marker=dict(color=color_map['Closed'], size=10),
        name='First closed model',
        text=time_deltas[bench]['Closed system'],
        showlegend=(idx == 0),
    ), row=1, col=idx + 1)
    fig.add_trace(go.Scatter(
        y=time_deltas[bench]['Closed performance level'],
        x=time_deltas[bench]['Lag (months)'],
        mode='markers',
        marker=dict(color=color_map['Open'], size=10),
        name='First open model',
        text=time_deltas[bench]['Open system'],
        showlegend=(idx == 0),
    ), row=1, col=idx + 1)

fig.update_yaxes(
    tickvals=performance_levels,
    title='Accuracy threshold (%)',
    showgrid=False,
)
fig.update_xaxes(
    title='Lag (months)',
    showgrid=False,
)

fig.update_layout(
    width=800,
    height=400,
    title='Open models usually take 3 to 12 months longer to pass an accuracy threshold',
    yaxis=dict(tickvals=performance_levels, title=None),
    template='plotly_white',
    # showlegend=False,
    legend=dict(
        orientation='h',
        yanchor='top',
        y=-0.25,
        xanchor='left',
        x=0,
    )
)

if save:
    save_plot(fig, results_dir, f"benchmark_time_deltas_{'_'.join(benchmarks_to_analyze)}_all")

# Show the plot
fig.show()

In [49]:
all_lags = []
for bench in benchmarks_to_analyze:
    all_lags.extend(time_deltas[bench]['Lag (months)'])
all_lags = np.array(all_lags)
print(f"Mean lag: {np.mean(all_lags):.0f} months")
print(f"Median lag: {np.median(all_lags):.0f} months")
print(f"75% of lags are less than {np.percentile(all_lags, 75):.0f} months")
print(f"25% of lags are less than {np.percentile(all_lags, 25):.0f} months")


Mean lag: 8 months
Median lag: 6 months
75% of lags are less than 14 months
25% of lags are less than 1 months


In [50]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=all_lags, nbinsx=20, opacity=0.75))
fig.update_layout(
    title='Distribution of lag (months)',
    xaxis_title='Lag (months)',
    yaxis_title='Count',
    template='plotly_white',
)
fig.show()


In [51]:
# Weighted average of compute lag and benchmark lags
compute_lag = 15  # Check this number is up to date with compute_analysis.ipynb
# Compute lag is weighted as much as the benchmark lags combined
weights = np.concatenate([np.ones(len(all_lags)), [compute_lag]])
all_lags_with_compute = np.concatenate([all_lags, [compute_lag]])
weighted_avg_lag = np.sum(all_lags_with_compute * weights) / np.sum(weights)
print(f"Weighted average lag: {weighted_avg_lag:.0f} months")

Weighted average lag: 10 months


## Plot benchmarks over time

### All together

In [52]:
fig = make_subplots(rows=2, cols=2, subplot_titles=benchmarks_to_analyze, vertical_spacing=0.2, horizontal_spacing=0.2)

# Define x limits for each subplot
x_limits = {
  'MMLU': [datetime(2019, 1, 1), datetime(2025, 1, 1)],
  'BBH': [datetime(2019, 1, 1), datetime(2025, 1, 1)],
  'GSM1k': [datetime(2019, 1, 1), datetime(2025, 1, 1)],
  'GPQA': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
  'SEAL Coding': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
  'SEAL Math': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
  'LMSys Elo': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
}

for i, bench in enumerate(benchmarks_to_analyze):
  for category in ['Open', 'Closed']:
    category_df = pd.DataFrame(max_rows_by_bench[bench][category])

    fig.append_trace(
        go.Scatter(
            x=category_df['Date'],
            y=100 * (1 - np.exp(-category_df['Score'])),
            mode='lines',
            line=dict(color=color_map[category], shape='hv'),
            text=category_df['System'],
            name=category,
            legendgroup=category,
            showlegend=True if i == 0 else False
        ),
        row=i//2 + 1, col=i%2 + 1
    )

  # Update x and y axes for this subplot
  dtick = "M12" if bench in ["GPQA"] else "M24"
  fig.update_xaxes(
    title_text="Model publication date" if i//2 + 1 == 2 else None,
    range=[x_limits[bench][0], x_limits[bench][1]],  # Set x limits
    row=i//2 + 1,
    col=i%2 + 1,
    dtick=dtick,  # Set tick marks to every 12 months
    # tickformat="%Y",  # Display only the year
    # ticklabelmode="period",  # Ensure labels are centered on the year
    tickangle=0  # Make tick labels horizontal
  )

  if i%2 + 1 == 1:
    fig.update_yaxes(title_text="Accuracy (%)", row=i//2 + 1, col=i%2 + 1)

# Title
fig.update_layout(title_text="Open models trail closed models on key benchmarks")

# Improve the layout
fig.update_layout(
  width=600,
  height=400,
  legend_title="Model accessibility",
  font=dict(size=12),
  hovermode="closest",
  template="plotly_white",
)

# Margins
fig.update_layout(
  margin=dict(l=10, r=10, t=70, b=80)
)

footnote = None
if exclude_all_llamas:
    footnote = '*All Llama models excluded'
elif exclude_big_llama:
    footnote = '*Llama 3.1 405B excluded'
    
if footnote:
    fig.add_annotation(
        showarrow=False,
        text=footnote,
        font=dict(size=10), 
        xref="paper",
        x=0,
        yref="paper",
        y=-0.2,
        xanchor="left",
        yanchor="top",
    )

# Save the plot
if save:
  save_plot(fig, results_dir, f"benchmark_dates_{'_'.join(benchmarks_to_analyze)}")

# Show the plot
fig.show()

### One at a time

In [53]:
# Define x limits for each subplot
x_limits = {
  'MMLU': [datetime(2019, 6, 1), datetime(2025, 1, 1)],
  'BBH': [datetime(2018, 1, 1), datetime(2026, 1, 1)],
  'GSM1k': [datetime(2019, 1, 1), datetime(2027, 1, 1)],
  'GPQA': [datetime(2022, 1, 1), datetime(2026, 1, 1)],
  'SEAL Coding': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
  'SEAL Math': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
  'LMSys Elo': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
}

for bench in benchmarks_to_analyze:
    fig = go.Figure()

    if save:
        pd.DataFrame(discrete_lags[bench]).to_csv(f"{results_dir}/plot_data/benchmark_lags_{bench}.csv", index=False)

    for category in ['Open', 'Closed']:
        category_df = pd.DataFrame(max_rows_by_bench[bench][category])

        category_df['Accuracy (%)'] = 100 * (1 - np.exp(-category_df['Score']))
        # Only include the columns we need
        category_df = category_df[['Date', 'System', 'Accuracy (%)']]
        if save:
            category_df.to_csv(f"{results_dir}/plot_data/benchmark_dates_{bench}_{category}.csv", index=False)

        fig.add_trace(go.Scatter(
            x=category_df['Date'],
            y=category_df['Accuracy (%)'],
            mode='lines+markers',
            line=dict(color=color_map[category], shape='hv', width=3),
            text=category_df['System'],
            name=category,
        ))

        # Annotate models where we want to highlight the lag
        # for i in range(len(discrete_lags[bench]['Closed performance level'])):
        #     fig.add_trace(go.Scatter(
        #         x=[discrete_lags[bench][f'{category} date'][i]],
        #         y=[discrete_lags[bench][f'{category} performance level'][i]],
        #         mode='text',
        #         text=discrete_lags[bench][f'{category} system'][i],
        #         textposition=('top left' if category == 'Closed' else 'bottom right'),
        #         showlegend=False,
        #     ))


    # Update x and y axes for this subplot
    dtick = "M12"
    fig.update_xaxes(
        title_text="Model publication date" if i//2 + 1 == 2 else None,
        range=[x_limits[bench][0], x_limits[bench][1]],  # Set x limits
        dtick=dtick,  # Set tick marks to every 12 months
        # tickformat="%Y",  # Display only the year
        # ticklabelmode="period",  # Ensure labels are centered on the year
        tickangle=0,  # Make tick labels horizontal
    )
    fig.update_yaxes(title_text="Accuracy (%)")

    # Title
    fig.update_layout(title_text=f"Top-performing open and closed AI models on {bench}")

    # Improve the layout
    fig.update_layout(
    width=600,
    height=400,
    legend_title="Model accessibility",
    font=dict(size=12),
    hovermode="closest",
    template="plotly_white",
    )

    footnote = None
    if exclude_all_llamas:
        footnote = '*All Llama models excluded'
    elif exclude_big_llama:
        footnote = '*Llama 3.1 405B excluded'
        
    if footnote:
        fig.add_annotation(
            showarrow=False,
            text=footnote,
            font=dict(size=10), 
            xref="paper",
            x=0,
            yref="paper",
            y=-0.2,
            xanchor="left",
            yanchor="top",
        )

    # Save the plot
    if save:
        save_plot(fig, results_dir, f"benchmark_dates_{bench}")

    # Show the plot
    fig.show()

## Pedagogical plot for measuring lag

In [54]:
fig = go.Figure()

# Define x limits for each subplot
x_limits = {
  'MMLU': [datetime(2022, 1, 1), datetime(2026, 1, 1)],
  'BBH': [datetime(2019, 1, 1), datetime(2025, 1, 1)],
  'GSM1k': [datetime(2019, 1, 1), datetime(2025, 1, 1)],
  'GPQA': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
  'SEAL Coding': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
  'SEAL Math': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
  'LMSys Elo': [datetime(2023, 1, 1), datetime(2025, 1, 1)],
}

bench = 'MMLU'

closed_model = 'GPT-4 (original)'
open_model = 'Llama 3.1 405B'
closed_date = pd.to_datetime(df[df['System'] == closed_model]['Date'].values[0])
closed_score = df[df['System'] == closed_model][bench].values[0]
open_date = pd.to_datetime(df[df['System'] == open_model]['Date'].values[0])
open_score = df[df['System'] == open_model][bench].values[0]

# fig.add_trace(go.Scatter(
#     x=[closed_date],
#     y=[100*closed_score],
#     mode='markers+text',
#     marker=dict(size=10, color=color_map['Closed']),
#     showlegend=False,
# ))
# fig.add_trace(go.Scatter(
#     x=[open_date],
#     y=[100*open_score],
#     mode='markers+text',
#     marker=dict(size=10, color=color_map['Open']),
#     showlegend=False,
# ))
line_color = 'rgb(150, 150, 150)'
# Horizontal line at each score level
fig.add_hline(y=100 * closed_score, line_color=line_color, line_width=1)
# fig.add_hline(y=100 * open_score, line_dash="dash", line_color="black", line_width=1)
# Vertical line at each date
fig.add_vline(x=closed_date, line_color=line_color, line_width=1)
fig.add_vline(x=open_date, line_color=line_color, line_width=1)
# Horizontal line shape with arrows near the x-axis, in between the vertical lines
fig.add_shape(
    type='line',
    x0=closed_date,
    y0=25,
    x1=open_date,
    y1=25,
    line=dict(color=line_color, width=2),
)
# Annotate line shape with lag
example_lag = (open_date - closed_date).days / 365 * 12
fig.add_annotation(
    x=closed_date + (open_date - closed_date) / 2,
    y=28,
    text=f'{example_lag:.0f} months',
    xref="x",
    yref="y",
    showarrow=False,
)

for category in ['Open', 'Closed']:
    category_df = pd.DataFrame(max_rows_by_bench[bench][category])

    fig.add_trace(go.Scatter(
        x=category_df['Date'],
        y=100 * (1 - np.exp(-category_df['Score'])),
        mode='lines',
        line=dict(color=color_map[category], shape='hv', width=3),
        text=category_df['System'],
        name=category,
    ))

fig.add_annotation(
    x=closed_date,
    y=100 * closed_score,
    text=closed_model,
    xref="x",
    yref="y",
    showarrow=False,
    align="left",
    xanchor="right",
    yanchor="bottom",
)
fig.add_annotation(
    x=open_date,
    y=100 * open_score,
    text=open_model,
    xref="x",
    yref="y",
    showarrow=False,
    align="left",
    xanchor="left",
    yanchor="bottom",
)

# Update x and y axes for this subplot
dtick = "M12"
fig.update_xaxes(
    title_text="Model publication date" if i//2 + 1 == 2 else None,
    range=[x_limits[bench][0], x_limits[bench][1]],  # Set x limits
    dtick=dtick,  # Set tick marks to every 12 months
    # tickformat="%Y",  # Display only the year
    # ticklabelmode="period",  # Ensure labels are centered on the year
    tickangle=0,  # Make tick labels horizontal
)
fig.update_yaxes(title_text="MMLU accuracy (%)")

# Title
fig.update_layout(title_text="Top-performing open and closed AI models on MMLU")

# Improve the layout
fig.update_layout(
  width=600,
  height=400,
  legend_title="Model accessibility",
  font=dict(size=12),
  hovermode="closest",
  template="plotly_white",
)

footnote = None
if exclude_all_llamas:
    footnote = '*All Llama models excluded'
elif exclude_big_llama:
    footnote = '*Llama 3.1 405B excluded'
    
if footnote:
    fig.add_annotation(
        showarrow=False,
        text=footnote,
        font=dict(size=10), 
        xref="paper",
        x=0,
        yref="paper",
        y=-0.2,
        xanchor="left",
        yanchor="top",
    )

# Save the plot
if save:
  save_plot(fig, results_dir, f"lag_example_{bench}")

# Show the plot
fig.show()

## Compute

In [55]:
fig = make_subplots(rows=2, cols=2, subplot_titles=benchmarks_to_analyze, vertical_spacing=0.15)

# Define x limits for each subplot
x_limits = {
    'MMLU': [1e20, 1e26],
    'GSM1k': [1e20, 1e26],
    'GPQA': [1e23, 1e26],
    'BBH': [1e20, 1e26],
    'SEAL Math': [1e23, 1e26],
    'SEAL Coding': [1e23, 1e26],
    'LMSys Elo': [1e22, 1e26],
}

for i, bench in enumerate(benchmarks_to_analyze):
    plot_df = df[~(df['System'] == 'Random chance')].copy()
    if old_models_only:
        plot_df = plot_df[plot_df['Before November 2023'] == 'Before'].copy()
    elif new_models_only:
        plot_df = plot_df[plot_df['Before November 2023'] == 'After'].copy()
    if non_suspects_only:
        if bench == 'GPQA':
            # GPQA was released November 20, 2023
            old_df = plot_df[plot_df['Before November 2023'] == 'Before'].copy()
            new_df = plot_df[plot_df['Before November 2023'] == 'After'].copy()
            new_df = new_df[new_df['Trust in benchmark results'] >= 0]
            plot_df = pd.concat([old_df, new_df])
        elif bench == 'MMLU':
            plot_df = plot_df[plot_df['Trust in benchmark results'] >= 0].copy()
    elif trusted_only:
        plot_df = plot_df[plot_df['Trust in benchmark results'] > 0].copy()

    for category in ['Open', 'Closed']:
        category_df = plot_df[plot_df['Open/Closed'] == category].copy()
        category_df['Accuracy (%)'] = 100 * category_df[bench]

        if plot_log_error and bench_is_accuracy[bench]:
            y = -np.log(1 - category_df[bench])
        else:
            y = 100 * category_df[bench]

        if save:
            category_df[['System', 'Training compute (FLOP)', 'Accuracy (%)']].to_csv(f"{results_dir}/plot_data/benchmark_compute_{bench}_{category}.csv", index=False)

        fig.append_trace(
            go.Scatter(
                x=category_df['Training compute (FLOP)'],
                y=y,
                mode='markers',
                marker=dict(color=color_map[category]),
                text=category_df['System'],
                name=category,
                legendgroup=category,
                showlegend=True if i == 0 else False
            ),
            row=i // 2 + 1, col=i % 2 + 1
        )

    # Update x and y axes for this subplot
    fig.update_xaxes(
        title_text="Training compute (FLOP)" if i // 2 + 1 == 2 else None,
        type='log',
        range=[np.log10(x_limits[bench][0]), np.log10(x_limits[bench][1])],  # Set x limits
        tickmode='linear',
        dtick=2,  # This sets ticks at every two powers of 10
        row=i // 2 + 1,
        col=i % 2 + 1
    )

    if i % 2 + 1 == 1:
        fig.update_yaxes(title_text="Accuracy (%)", row=i // 2 + 1, col=i % 2 + 1)

# Improve the layout
fig.update_layout(
    template='plotly_white',
    width=600,
    height=400,
    # legend_title="Model accessibility",
    font=dict(size=12),
    hovermode="closest",
)

# Margins
fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0)
)

# Save the plot
if save:
    save_plot(fig, results_dir, 'benchmark_compute')

# Show the plot
fig.show()

In [56]:
bench = 'MMLU'

In [57]:
reg_df = df[~(df['System'] == 'Random chance')]
if non_suspects_only:
  if bench == 'GPQA':
    # GPQA was released November 20, 2023
    old_df = reg_df[reg_df['Before November 2023'] == 'Before']
    new_df = reg_df[reg_df['Before November 2023'] == 'After']
    new_df = new_df[new_df['Trust in benchmark results'] >= 0]
    reg_df = pd.concat([old_df, new_df])
  elif bench == 'MMLU':
    reg_df = reg_df[reg_df['Trust in benchmark results'] >= 0]
elif trusted_only:
  reg_df = reg_df[reg_df['Trust in benchmark results'] > 0]

In [58]:
random_chance_level = df.loc[df["System"] == "Random chance", bench].values[0]
# Filter out models that are not far above random chance level
# This is a heuristic to find the changepoint
filtered_reg_df = reg_df.loc[reg_df[bench] > random_chance_level + 0.05].copy()
filtered_reg_df['log_compute'] = np.log10(filtered_reg_df['Training compute (FLOP)'])
filtered_reg_df[bench + '_log_error'] = -np.log(1 - filtered_reg_df[bench])
filtered_reg_df.dropna(subset=['log_compute', bench + '_log_error'], inplace=True)

In [59]:
filtered_reg_df.loc[:, 'year'] = datetime_to_float_year(filtered_reg_df['Date'])

In [60]:
# Single fit for all data
model = fit_ols_regression(filtered_reg_df, ['log_compute', 'year'], bench + '_log_error')
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         MMLU_log_error   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.890
Method:                 Least Squares   F-statistic:                     195.9
Date:                Tue, 05 Nov 2024   Prob (F-statistic):           3.12e-23
Time:                        09:21:24   Log-Likelihood:                 21.431
No. Observations:                  49   AIC:                            -36.86
Df Residuals:                      46   BIC:                            -31.19
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    -344.0901     57.040     -6.032      0.000    -458.905    -229.275
log_compute     0.4395      0.028     15.611      0.000       0.383       0.496
year            0.1654      0.028      5.843      0.000       0.108       0.222
==============================================================================
Omnibus:                        1.426   Durbin-Watson:                   1.433
Prob(Omnibus):                  0.490   Jarque-Bera (JB):                0.786
Skew:                           0.287   Prob(JB):                        0.675
Kurtosis:                       3.236   Cond. No.                     5.01e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.01e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [61]:
model.params.index

Index(['Intercept', 'log_compute', 'year'], dtype='object')

In [62]:
get_predictions(model, filtered_reg_df, ['log_compute', 'year'])

array([0.81434632, 0.80638696, 0.83132637, 1.10486387, 0.73948667,
       1.35046281, 1.2850296 , 0.90067025, 1.96239956, 1.14629742,
       0.79653611, 0.79787486, 1.31852149, 1.6768343 , 0.3659553 ,
       1.41022501, 1.64864726, 1.69698771, 1.62755756, 2.01419568,
       1.67117169, 0.59902681, 0.84093469, 0.97046828, 0.48086266,
       0.79753631, 0.98105351, 1.11889686, 0.67937216, 1.69180854,
       1.90415027, 0.82219318, 0.51360842, 1.85370611, 1.85370611,
       0.44787079, 1.11627011, 0.70476383, 1.50148341, 0.74664771,
       0.68249532, 0.63235476, 0.41243636, 1.10486387, 1.22735714,
       0.76029253, 0.93725674, 0.77523145, 1.39994526])

In [63]:
def no_split(df, filter_threshold=None):
  return {'All': df}


def open_closed_split(df, filter_threshold=None):
  open_df = df[df['Open/Closed'] == 'Open']
  closed_df = df[df['Open/Closed'] == 'Closed']
  return {'Open': open_df, 'Closed': closed_df}


def new_old_split(df, date):
  new_df = df[df['Date'] >= date]
  old_df = df[df['Date'] < date]
  return {'Before': old_df, 'After': new_df}


def combined_rsquared(xs, ys, models):
    y_true = np.concatenate(ys)
    y_mean = np.mean(y_true)
    y_pred = []
    for i, model in enumerate(models):
        X = sm.add_constant(xs[i])
        y_pred.append(model.predict(X))
    y_pred = np.concatenate(y_pred)
    
    sst = np.sum((y_true - y_mean)**2)
    ssr = np.sum((y_true - y_pred)**2)
    r_squared = 1 - (ssr / sst)
    return r_squared


def combined_bic(xs, ys, models):
  y = np.concatenate(ys)
  y_pred = []
  total_params = 0
  for i, model in enumerate(models):
    X = sm.add_constant(xs[i])
    y_pred.append(model.predict(X))
    total_params += len(model.params)
  y_pred = np.concatenate(y_pred)
  n = len(y)
  rss = np.sum((y - y_pred)**2)
  ll = -n/2 * (1 + np.log(2*np.pi) + np.log(rss/n))
  bic = -2 * ll + total_params * np.log(n)
  return bic


# K-Fold Cross Validation
def perform_cross_validation(df, filter_fn, features, bench, k=10, random_state=42, filter_threshold=None):
  kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
  folds_mses = []
  for train_index, test_index in kf.split(df):
    train_df, test_df = df.iloc[train_index], df.iloc[test_index]
    train_dfs = filter_fn(train_df, filter_threshold)
    test_dfs = filter_fn(test_df, filter_threshold)

    # Fit the models on the training set
    submodels = {}
    for category, train_df in train_dfs.items():
      model = fit_ols_regression(train_df, features, bench + '_log_error')
      submodels[category] = model

    # Predict on the test set
    residuals = []
    for i, (category, test_df) in enumerate(test_dfs.items()):
      predicted = get_predictions(submodels[category], test_df, features)
      residuals.append(predicted - test_df[bench + '_log_error'])
    residuals = np.concatenate(residuals)
    mse = np.mean(residuals**2)
    folds_mses.append(mse)

  return np.array(folds_mses)


def regression_with_results(df, filter_fn, features, bench, filter_threshold=None):
  dfs = filter_fn(df, filter_threshold)
  submodels = {category: fit_ols_regression(df, features, bench + '_log_error') for category, df in dfs.items()}
  mses = perform_cross_validation(df, filter_fn, features, bench, filter_threshold=filter_threshold)
  bic = combined_bic(
      [dfs[category][features] for category in dfs],
      [dfs[category][bench + '_log_error'] for category in dfs],
      [submodels[category] for category in dfs],
  )
  rsquared = combined_rsquared(
      [dfs[category][features] for category in dfs],
      [dfs[category][bench + '_log_error'] for category in dfs],
      [submodels[category] for category in dfs],
  )
  return {'mses': mses, 'bic': bic, 'rsquared': rsquared, 'submodels': submodels}


def boostrapped_regression_with_results(df, filter_fn, features, bench, filter_threshold=None):
  bootstrap_results = []
  for i in tqdm(range(1000)):
    resampled_df = df.sample(frac=1, replace=True, random_state=rng)
    bootstrap_results.append(regression_with_results(resampled_df, filter_fn, features, bench, filter_threshold))
  return bootstrap_results


In [64]:
if bootstrap_compute_performance_regression:
    bootstrap_results = {}
    bootstrap_results['no_split'] = boostrapped_regression_with_results(filtered_reg_df, no_split, ['log_compute'], bench)
    bootstrap_results['no_split_year'] = boostrapped_regression_with_results(filtered_reg_df, no_split, ['log_compute', 'year'], bench)
    bootstrap_results['open_closed_split'] = boostrapped_regression_with_results(filtered_reg_df, open_closed_split, ['log_compute'], bench)
    bootstrap_results['open_closed_split_year'] = boostrapped_regression_with_results(filtered_reg_df, open_closed_split, ['log_compute', 'year'], bench)

    ci = [5, 95]
    for split, result in bootstrap_results.items():
        print(f"Results for {split}:")
        bootstrapped_mses = np.array([r['mses'] for r in result])
        bootstrapped_mean_mses = np.mean(bootstrapped_mses, axis=1)
        print(f"Mean k-fold MSE: {bootstrapped_mean_mses.mean():.3f} ({np.percentile(bootstrapped_mean_mses, ci)})")
        print(f"BIC: {np.mean([r['bic'] for r in result]):.4f} ({np.percentile([r['bic'] for r in result], ci)})")
        print(f"R-squared: {np.mean([r['rsquared'] for r in result]):.4f} ({np.percentile([r['rsquared'] for r in result], ci)})")
        print("--------------------")

    ci = [5, 95]
    ci_width = ci[1] - ci[0]
    results_list = []
    splits_names = {
        'no_split': 'Compute only',
        'no_split_year': 'Compute + date',
        'open_closed_split': 'Compute + open/closed',
        'open_closed_split_year': 'Compute + date + open/closed'
    }
    for split, result in bootstrap_results.items():
        bootstrapped_mses = np.array([r['mses'] for r in result])
        bootstrapped_mean_mses = np.mean(bootstrapped_mses, axis=1)
        mean_mse = round(bootstrapped_mean_mses.mean(), 3)
        mean_mse_ci_lower = round(np.percentile(bootstrapped_mean_mses, ci[0]), 3)
        mean_mse_ci_upper = round(np.percentile(bootstrapped_mean_mses, ci[1]), 3)
        bic_mean = round(np.mean([r['bic'] for r in result]), 0)
        bic_ci_lower = round(np.percentile([r['bic'] for r in result], ci[0]), 0)
        bic_ci_upper = round(np.percentile([r['bic'] for r in result], ci[1]), 0)
        rsquared_mean = round(np.mean([r['rsquared'] for r in result]), 4)
        rsquared_ci_lower = round(np.percentile([r['rsquared'] for r in result], ci[0]), 3)
        rsquared_ci_upper = round(np.percentile([r['rsquared'] for r in result], ci[1]), 3)
        
        results_list.append({
            'Regress on': splits_names[split],
            'Mean k-fold MSE': mean_mse,
            f'Mean k-fold MSE {ci_width}% CI': (mean_mse_ci_lower, mean_mse_ci_upper),
            'BIC Mean': bic_mean,
            f'BIC {ci_width}% CI': (bic_ci_lower, bic_ci_upper),
            'R-squared Mean': rsquared_mean,
            f'R-squared {ci_width}% CI': (rsquared_ci_lower, rsquared_ci_upper)
        })

    results_df = pd.DataFrame(results_list)
    display(results_df)
    split1 = 'open_closed_split'
    split2 = 'no_split_year'
    for metric in ['bic', 'rsquared']:
        diff = np.array([r2[metric] - r1[metric] for r1, r2 in zip(bootstrap_results[split1], bootstrap_results[split2])])
        print(f"{split2} - {split1} {metric}: {diff.mean()} (90% CI: {np.percentile(diff, ci)})")

In [65]:
# Without bootstrapping
results = {}
results['no_split'] = regression_with_results(filtered_reg_df, no_split, ['log_compute'], bench)
results['no_split_year'] = regression_with_results(filtered_reg_df, no_split, ['log_compute', 'year'], bench)
results['open_closed_split'] = regression_with_results(filtered_reg_df, open_closed_split, ['log_compute'], bench)
results['open_closed_split_year'] = regression_with_results(filtered_reg_df, open_closed_split, ['log_compute', 'year'], bench)
for split, result in results.items():
  print(f"Results for {split}:")
  print(f"Mean k-fold MSE: {np.mean(result['mses']):.3f} ({np.std(result['mses']):.3f})")
  print(f"BIC: {result['bic']:.4f}")
  print(f"R-squared: {result['rsquared']:.4f}")
  print("--------------------")

Results for no_split:
Mean k-fold MSE: 0.047 (0.029)
BIC: -7.8757
R-squared: 0.8169
--------------------
Results for no_split_year:
Mean k-fold MSE: 0.029 (0.018)
BIC: -31.1875
R-squared: 0.8949
--------------------
Results for open_closed_split:
Mean k-fold MSE: 0.044 (0.033)
BIC: -5.0697
R-squared: 0.8346
--------------------
Results for open_closed_split_year:
Mean k-fold MSE: 0.032 (0.024)
BIC: -25.9424
R-squared: 0.9078
--------------------


In [66]:
filtered_reg_df['System'].to_list()

['BLOOM-176B',
 'BloombergGPT',
 'Chinchilla 70B',
 'code-davinci-002',
 'DeepSeek-7B',
 'DeepSeek-Coder-V2',
 'DeepSeek-V2',
 'Falcon 40B',
 'Gemini Ultra',
 'Gemma 2 9B',
 'GLM 130B',
 'Gopher 280B',
 'GPT-3.5-turbo-16k',
 'GPT-4 (original)',
 'GPT-NeoX 20B',
 'Grok-1',
 'Inflection-2',
 'Inflection-2.5',
 'Llama 3 70B',
 'Llama 3.1 405B',
 'Llama 3.1 70B',
 'LLaMa-1 13B',
 'LLaMa-1 33B',
 'LLaMa-1 65B',
 'LLaMa-1 7B',
 'LLaMa-2 13B',
 'LLaMa-2 34B',
 'LLaMa-2 70B',
 'LLaMa-2 7B',
 'Mistral Large',
 'Mistral Large 2',
 'MPT 30B',
 'MPT 7B',
 'Nemotron-4-340B Base',
 'Nemotron-4-340B Instruct',
 'OPT-66B',
 'PaLM 540B',
 'PaLM 63B',
 'PaLM-2',
 'Qwen 7B',
 'StableLM-3B-4E1T',
 'StableLM-alpha-7b-v2',
 'text-davinci-001',
 'text-davinci-002',
 'text-davinci-003',
 'XVerse-13B',
 'XVerse-13B-2',
 'Yi-6B',
 'Yi-Large']

In [67]:
fig = go.Figure()

for category, color in [('Open', 'blue'), ('Closed', 'orange')]:
    df_category = filtered_reg_df[filtered_reg_df['Open/Closed'] == category].copy()

    df_category['Accuracy (%)'] = 100 * df_category[bench]
    if save:
        df_category[['System', 'Training compute (FLOP)', 'Accuracy (%)']].to_csv(f"{results_dir}/plot_data/{bench}_compute_{category}.csv", index=False)

    fig.add_trace(
        go.Scatter(
            x=10 ** df_category['log_compute'],
            y=100 * df_category[bench],
            mode='markers',
            name=f'{category}',
            marker=dict(color=color, opacity=0.5),
            text=df_category['System']
        ),
    )

# Make a data frame for the annotations
annotations_df = filtered_reg_df[filtered_reg_df['System'].isin(['DeepSeek-V2', 'PaLM-2','Gemma 2 9B', 'PaLM 540B'])].copy()
annotations_df['Accuracy (%)'] = 100 * annotations_df[bench]
if save:
    annotations_df[['System', 'Training compute (FLOP)', 'Accuracy (%)']].to_csv(f"{results_dir}/plot_data/{bench}_compute_annotations.csv", index=False)

fig.add_annotation(
    x=24,
    y=78.5,
    text="DeepSeek V2",
    xref="x",
    yref="y",
    showarrow=True,
    arrowhead=2,
    font=dict(size=10),
    align="left",
    xanchor="right",
    ax=-10,
    ay=-10
)
fig.add_annotation(
    x=24.87,
    y=78.3,
    text="PaLM 2",
    xref="x",
    yref="y",
    showarrow=True,
    arrowhead=2,
    font=dict(size=10),
    align="left",
    xanchor="left",
    ax=10,
    ay=10,
)
fig.add_annotation(
    x=23.64,
    y=71.3,
    text="Gemma 2 9B",
    xref="x",
    yref="y",
    showarrow=True,
    arrowhead=2,
    font=dict(size=10),
    align="left",
    xanchor="right",
    ax=-10,
    ay=-10
)
fig.add_annotation(
    x=24.4,
    y=71.3,
    text="PaLM 540B",
    xref="x",
    yref="y",
    showarrow=True,
    arrowhead=2,
    font=dict(size=10),
    align="left",
    xanchor="left",
    ax=10,
    ay=10
)
fig.add_shape(
    type="line",
    x0=10 ** 24.0,
    y0=78.5,
    x1=10 ** 24.87,
    y1=78.5,
    line=dict(color="black", width=1)
)
fig.add_annotation(
    x=(24.87 + 24.0) / 2,
    y=78.5 + 3,
    text="7x",
    showarrow=False,
    font=dict(size=10),
    align="center",
)
fig.add_shape(
    type="line",
    x0=10 ** 23.64,
    y0=71.3,
    x1=10 ** 24.4,
    y1=71.3,
    line=dict(color="black", width=1)
)
fig.add_annotation(
    x=(24.4 + 23.64) / 2,
    y=71.3 + 3,
    text="6x",
    showarrow=False,
    font=dict(size=10),
    align="center",
)

# Update layout
fig.update_layout(
    width=600,
    height=300,
    title_text=f"Some open LLMs use less compute to match closed LLMs",
    showlegend=True,
    legend=dict(
        title="Model accessibility"
    ),
    margin=dict(t=50, b=20, l=40, r=40),
    template='plotly_white'
)

fig.update_xaxes(
    title_text="Training compute (FLOP)" if i // 2 + 1 == 2 else None,
    type='log',
    range=[22, 26],  # Set x limits
    tickmode='linear',
    dtick=2,  # This sets ticks at every two powers of 10
)
fig.update_yaxes(
    title_text="MMLU accuracy (%)",
    range=[25, 100],
)

# Show the plot
fig.show()

# Save the plot
if save:
    save_plot(fig, results_dir, f'{bench}_compute')

In [68]:
regression_df = filtered_reg_df.copy()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=10**filtered_reg_df['log_compute'],
        y=filtered_reg_df[bench + '_log_error'],
        mode='markers',
        name='Data',
        marker=dict(color='gray', opacity=0.5),
        text=filtered_reg_df['System']
    ),
)

all_model = results['no_split']['submodels']['All']
y_pred = get_predictions(all_model, filtered_reg_df, ['log_compute'])
regression_df['Fit (compute)'] = y_pred

fig.add_trace(
    go.Scatter(
        x=10**filtered_reg_df['log_compute'],
        y=y_pred,
        mode='lines',
        name=f'Fit (compute)',
        text=filtered_reg_df['System'],
        marker=dict(color='black'),
    ),
)

# for category, color in [('Open', 'blue'), ('Closed', 'orange')]:
#     category_models = results['open_closed_split_year']['submodels']
#     model_category = category_models[category]
#     category_df = filtered_reg_df[filtered_reg_df['Open/Closed'] == category]
#     y_pred = get_predictions(model_category, category_df, ['log_compute', 'year'])
  
#     fig.add_trace(
#         go.Scatter(
#             x=category_df['log_compute'],
#             y=y_pred,
#             mode='markers',
#             name=f'Fit (compute, age, {category})',
#             text=category_df['System'],
#             marker=dict(color=color, opacity=0.5, symbol='square'),
#         ),
#     )

all_model = results['no_split_year']['submodels']['All']
y_pred = get_predictions(all_model, filtered_reg_df, ['log_compute', 'year'])
regression_df['Fit (compute, date)'] = y_pred

fig.add_trace(
    go.Scatter(
        x=10**filtered_reg_df['log_compute'],
        y=y_pred,
        mode='markers',
        name='Fit (compute, date)',
        text=filtered_reg_df['System'],
        marker=dict(color='green', opacity=0.5),
    ),
)

# Update layout
fig.update_layout(
    width=600,
    height=250,
    title_text=f"Regression on {bench} score vs. training compute",
    showlegend=True,
    # legend=dict(
    #     orientation="h",
    #     yanchor="bottom",
    #     y=-0.4,
    #     xanchor="left",
    #     x=0,
    # ),
    margin=dict(t=50, b=20, l=40, r=40),
    template='plotly_white'
)

fig.update_xaxes(title_text="Training compute (FLOP)", type='log')
fig.update_yaxes(title_text="Negative log of error rate")

# Show the plot
fig.show()

# Save the plot
if save:
    save_plot(fig, results_dir, f'{bench}_regression_models_comparison')
    regression_df[['System', 'Training compute (FLOP)', 'Fit (compute)', 'Fit (compute, date)', bench + '_log_error']].to_csv(f"{results_dir}/plot_data/{bench}_regression_models_comparison.csv", index=False)

In [69]:
fig = go.Figure()

category = 'Open'
color = 'blue'
category_models = results['open_closed_split_year']['submodels']
model_category = category_models[category]
category_df = filtered_reg_df[filtered_reg_df['Open/Closed'] == category]
y_pred = get_predictions(model_category, category_df, ['log_compute', 'year'])

fig.add_trace(
    go.Scatter(
        x=category_df['log_compute'],
        y=y_pred - category_df[bench + '_log_error'],
        mode='markers',
        name=f'Fit (compute, age, {category})',
        marker=dict(color=color, opacity=0.5),
    ),
)

all_model = results['no_split_year']['submodels']['All']
y_pred = get_predictions(all_model, category_df, ['log_compute', 'year'])

fig.add_trace(
  go.Scatter(
    x=category_df['log_compute'],
    y=y_pred - category_df[bench + '_log_error'],
    mode='markers',
    name='Fit (compute, age)',
    marker=dict(color='green', opacity=0.5),
  ),
)

# Update layout
fig.update_layout(
  width=600,
  height=300,
  title_text=f"{bench} score vs log compute",
  showlegend=True,
  # legend=dict(
  #   orientation="h",
  #   yanchor="bottom",
  #   y=-0.4,
  #   xanchor="left",
  #   x=0,
  # ),
  margin=dict(t=50, b=20, l=40, r=40)
)

fig.update_xaxes(title_text="Log Compute", range=[22, 26])
fig.update_yaxes(title_text="Residuals")

# Show the plot
fig.show()

# Save the plot
if save:
  save_plot(fig, results_dir, f'{bench}_regression_residuals_{category}')

In [70]:
# Calculate the average age of new models
# Need to convert the date to a number of days since the epoch
new_avg_age = (new_df['Date'] - new_df['Date'].min()).dt.days.mean()
old_avg_age = (old_df['Date'] - old_df['Date'].min()).dt.days.mean()
# Convert the average age back to date
new_avg_age_date = new_df['Date'].min() + pd.Timedelta(days=new_avg_age)
old_avg_age_date = old_df['Date'].min() + pd.Timedelta(days=old_avg_age)
print(f"Average age of new models: {new_avg_age_date}")
print(f"Average age of old models: {old_avg_age_date}")
print(f"Difference in average age: {new_avg_age_date - old_avg_age_date}")

Average age of new models: 2024-03-30 15:23:46.415094338
Average age of old models: 2022-10-26 20:34:17.142857136
Difference in average age: 520 days 18:49:29.272237202
